# Setting of stack phase 

At this phase we are going to set the stacked-phase dataset. This method is based just in apply method. It implies less memory.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from utils.trainFold import getVector

In [ ]:
import sys
sys.path.append('../dsbase/src/main')
from AdaBoostClassificationDSBase import AdaBoostClassificationDSBaseModel

## Loading the test dataset

In [ ]:
# cat datasets/test.csv | head -n +1001 > datasets/test_reduced.csv

In [ ]:
df = pd.read_csv('datasets/test.csv')

In [ ]:
# TODO Clean as training

## Defining the Fold X processing 

### Defining helping functions 

In [ ]:
def loadColumnCategoricalOrder(df, columns_categorical):
    columns_categorical_order_dict = {}
    for x in columns_categorical:
        columns_categorical_order_dict[x] = np.where(df.columns == x)[0][0]
    return columns_categorical_order_dict

In [ ]:
def loadColumnCategoricalVectors(fold_id, columns_categorical):
    columns_categorical_vectors_dict = {}
    out_path = 'models/fold' + str(fold_id)
    for c in columns_categorical:
        vec = np.load('models/fold' + str(fold_id) + "/" + c + ".sav.npy")
        columns_categorical_vectors_dict[c] = vec
    return columns_categorical_vectors_dict

In [ ]:
def loadModel(fold_id):
   # --------------------------------------
    # Load the i-th model and process
    print('   loading model ...')    
    model = AdaBoostClassificationDSBaseModel('AB2',None,None,None,None,None,None)
    model.load('models/fold' + str(fold_id))
    return model

In [ ]:
def calculate(x, cc, cc_o, cc_v, model):
    xnp = x.values
    acc=0
    for c in cc:
        index = cc_o[c] + acc
        vec = cc_v[c]
        new = getVector(xnp[index], vec)
        xnp = np.delete(xnp, index)
        xnp = np.insert(xnp, index, new)
        acc += (new.size - 1)
    pre_result = model.scalerX.transform(xnp.reshape(1,-1))
    result = model.model.predict_proba(pre_result)
    return result[0,1]

### calculating support variables

In [ ]:
df_w = df.drop(['MachineIdentifier','Census_ProcessorClass','Census_InternalBatteryType'], axis=1)
columns_categorical = df_w.select_dtypes(include=['object']).columns

In [ ]:
cc_order = loadColumnCategoricalOrder(df_w,columns_categorical)

In [ ]:
i = 2
print('-------- Process Fold ',i,' -------------------')
print('loading vectors ...')
cc_values_f = loadColumnCategoricalVectors(i,columns_categorical)
print('loading model ...')
model_f = loadModel(i)
print('applying folding prediction ...')
df['HasDetections'] = df_w.apply(func=calculate, axis=1, args=(columns_categorical, cc_order, cc_values_f, model_f))

## Save final dataset 

In [ ]:
df['MachineIdentifier','HasDetections'].to_csv('datasets/submission.csv')

# End of submission setting!! 